In [170]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import numpy as np

In [171]:
df_Beilagen_v1 = pd.read_excel('beilagen_Session7_tables.xlsx', header=None, names=['Transkribus_No', 'Image_No', 'Beilagen_No', 'Titel', 'Typ', 'Seite'])
df_Beilagen_v1.head()

C:\Users\georg\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Transkribus_No,Image_No,Beilagen_No,Titel,Typ,Seite
0,1,image_00000005.tif,1,Allerhöchste Thronrede.,NaN,"1, 2"
1,1,image_00000005.tif,NaN,"Forterhebung der Steuern und Abgaben, dann\nBe...",Regierungsvorlage.,3
2,1,image_00000005.tif,3,Finanzgesetz und Staatsooranschlag für das\nJa...,NaN,4—30
3,1,image_00000005.tif,NaN,Eröffnung von Nachtragscrediten für das\nJahr ...,NaN,31
4,1,image_00000005.tif,NaN,Ermächtigung zu einer Creditsoperation durch\n...,Regierungsvorlage\nund Motive.,32—43


In [172]:
df_Beilagen_v1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359 entries, 0 to 358
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Transkribus_No  359 non-null    int64 
 1   Image_No        359 non-null    object
 2   Beilagen_No     352 non-null    object
 3   Titel           359 non-null    object
 4   Typ             263 non-null    object
 5   Seite           355 non-null    object
dtypes: int64(1), object(5)
memory usage: 17.0+ KB


In [173]:
df_Beilagen_v1 = df_Beilagen_v1.drop('Transkribus_No', axis=1)

In [174]:
# Remove spaces at the beginning of the 'Titel' column
df_Beilagen_v1['Titel'] = df_Beilagen_v1['Titel'].astype(str).str.replace(r'^\s+', '', regex=True)
df_Beilagen_v1['Typ'] = df_Beilagen_v1['Typ'].astype(str).str.replace(r'^\s+', '', regex=True)

In [175]:
#remove line breaks
df_Beilagen_v1 = df_Beilagen_v1.replace(r'[\r\n]+', ' ', regex=True)

In [176]:
# Remove specific characters, e.g. "¬"
df_Beilagen_v1['Titel'] = df_Beilagen_v1['Titel'].astype(str).str.replace('¬', '', regex=False)
df_Beilagen_v1['Typ'] = df_Beilagen_v1['Typ'].astype(str).str.replace('¬', '', regex=False)

In [177]:
#supplement specific characters
df_Beilagen_v1['Titel'] = df_Beilagen_v1['Titel'].astype(str).str.replace('=', '-', regex=False)
df_Beilagen_v1['Typ'] = df_Beilagen_v1['Typ'].astype(str).str.replace('=', '-', regex=False)
df_Beilagen_v1['Titel'] = df_Beilagen_v1['Titel'].astype(str).str.replace('R. G. Bl.', 'RGBl.', regex=False)
df_Beilagen_v1['Typ'] = df_Beilagen_v1['Typ'].astype(str).str.replace('R. G. Bl.', 'RGBl.', regex=False)

In [178]:
df_Beilagen_v1.head()

,Image_No,Beilagen_No,Titel,Typ,Seite
0,image_00000005.tif,1,Allerhöchste Thronrede.,nan,"1, 2"
1,image_00000005.tif,NaN,"Forterhebung der Steuern und Abgaben, dann Bes...",Regierungsvorlage.,3
2,image_00000005.tif,3,Finanzgesetz und Staatsooranschlag für das Jah...,nan,4—30
3,image_00000005.tif,NaN,Eröffnung von Nachtragscrediten für das Jahr 1...,nan,31
4,image_00000005.tif,NaN,Ermächtigung zu einer Creditsoperation durch B...,Regierungsvorlage und Motive.,32—43


In [179]:
# Extract the first full number from "Seite" into a *new* column
df_Beilagen_v1['Seite_cleaned'] = (
    df_Beilagen_v1['Seite']
    .astype(str)                     # ensure it's string
    .str.extract(r'(\d+)')[0]        # extract the first digit sequence
)

# Convert to numeric (but keep NaN if no number found)
df_Beilagen_v1['Seite_cleaned'] = pd.to_numeric(df_Beilagen_v1['Seite_cleaned'], errors='coerce')

# Convert to int only where possible (optional: leaves others as NaN)
df_Beilagen_v1['Seite_cleaned'] = df_Beilagen_v1['Seite_cleaned'].astype('Int64')  # pandas nullable integer type

In [180]:
df_Beilagen_v1.tail()

,Image_No,Beilagen_No,Titel,Typ,Seite,Seite_cleaned
354,image_00000038.tif,353,"Aufhebung des Gesetzes vom 30. Juni 1868, RGBl...",Beschluß des Abge ordnetenhauses.,3075,3075
355,image_00000039.tif,354,Antrag des Abgeordneten Dr. Roser wegen Er las...,Bericht.,3076—3082,3076
356,image_00000039.tif,355,Polizeistrafrechtliche Bestimmungen wider Arbe...,Beschluß des Abge ordnetenhauses.,3083 — 3086,3083
357,image_00000039.tif,elztb,a) Hauptübersicht über den Stand der gesammten...,Vorlage der Staats schulden - Control- commiss...,3087—3093,3087
358,image_00000039.tif,or:,Thronrede Seiner k. und k. Apostolischen Majes...,nan,"3094, 3095.",3094


In [181]:
#herunterladen um in excel manuell die Beilagenzahlen zu korrigieren; einfach runterziehen und schauen ob jede Zelle nur eine Beilage ist; außerdem Seitenzahlen vergleichen --> vielleicht eine Art automatisierten check machen ob die zahlen aufsteigend sind und diese sonst farblich anzeigen
df_Beilagen_v1.to_excel('beilagen_Session7_tables_v2.xlsx', index=False)

In [182]:
df_Beilagen_v3 = pd.read_excel('beilagen_Session7_tables_v3_bereinigt.xlsx', names=['Image_No', 'Beilagen_No', 'Titel', 'Typ', 'Seite', 'Seite_cleaned'])
df_Beilagen_v3.head()

,Image_No,Beilagen_No,Titel,Typ,Seite,Seite_cleaned
0,image_00000005.tif,1,Allerhöchste Thronrede.,NaN,"1, 2",1.0
1,image_00000005.tif,2,"Forterhebung der Steuern und Abgaben, dann Bes...",Regierungsvorlage.,3,3.0
2,image_00000005.tif,3,Finanzgesetz und Staatsooranschlag für das Jah...,NaN,4—30,4.0
3,image_00000005.tif,4,Eröffnung von Nachtragscrediten für das Jahr 1...,NaN,31,31.0
4,image_00000005.tif,5,Ermächtigung zu einer Creditsoperation durch B...,Regierungsvorlage und Motive.,32—43,32.0


In [183]:
# Convert to int only where possible (optional: leaves others as NaN)
df_Beilagen_v3['Seite_cleaned'] = df_Beilagen_v3['Seite_cleaned'].astype('Int64')  # pandas nullable integer type

In [184]:
# Path to your uploaded file
file_path = 'Seitenkonkordanz_Session7.txt'

# Read the file line by line
with open(file_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()

pairs = []

# Regex pattern to capture value and inner number
pattern = r'<option value="(\d+)">(\d+)</option>'

for line in lines:
    matches = re.findall(pattern, line)
    # matches will be a list of tuples (value, inner_number)
    pairs.extend(matches)

# Create DataFrame
df_Seitenkonkordanz = pd.DataFrame(pairs, columns=['PageNumber', 'DocumentNumber'])

In [185]:
df_Seitenkonkordanz['PageNumber'] = 'https://alex.onb.ac.at/cgi-content/alex?aid=spa&datum=0007&page=' + df_Seitenkonkordanz['PageNumber'].astype(str)
df_Seitenkonkordanz

,PageNumber,DocumentNumber
0,https://alex.onb.ac.at/cgi-content/alex?aid=sp...,1
1,https://alex.onb.ac.at/cgi-content/alex?aid=sp...,2
2,https://alex.onb.ac.at/cgi-content/alex?aid=sp...,3
3,https://alex.onb.ac.at/cgi-content/alex?aid=sp...,4
4,https://alex.onb.ac.at/cgi-content/alex?aid=sp...,5
...,...,...
3073,https://alex.onb.ac.at/cgi-content/alex?aid=sp...,3091
3074,https://alex.onb.ac.at/cgi-content/alex?aid=sp...,3092
3075,https://alex.onb.ac.at/cgi-content/alex?aid=sp...,3093
3076,https://alex.onb.ac.at/cgi-content/alex?aid=sp...,3094


In [186]:
# Ensure both columns are the same type (string recommended)
df_Seitenkonkordanz['DocumentNumber'] = df_Seitenkonkordanz['DocumentNumber'].astype(str)
df_Beilagen_v3['Seite_cleaned'] = df_Beilagen_v3['Seite_cleaned'].astype(str)

# Merge the dataframes
df_Beilagen_v3 = df_Beilagen_v3.merge(
    df_Seitenkonkordanz[['DocumentNumber', 'PageNumber']],
    left_on='Seite_cleaned',
    right_on='DocumentNumber',
    how='left'  # keeps all rows in df_Beilagen_v3, unmatched get NaN
)


# Drop the redundant DocumentNumber column from the merge
df_Beilagen_v3.drop(columns=['DocumentNumber'], inplace=True)

In [187]:
#Punkte am Ende von Typ entfernen
df_Beilagen_v3['Typ'] = df_Beilagen_v3['Typ'].astype(str).str.replace(r'\.+$', '', regex=True)

In [188]:
#alle values unter 5 characters in Typ löschen
df_Beilagen_v3.loc[df_Beilagen_v3['Typ'].astype(str).str.len() < 5, 'Typ'] = np.nan

In [189]:
#substituting the null values in Typ with the antecedent value.
df_Beilagen_v3.Typ.fillna(method='ffill', inplace=True)
df_Beilagen_v3

,Image_No,Beilagen_No,Titel,Typ,Seite,Seite_cleaned,PageNumber
0,image_00000005.tif,1,Allerhöchste Thronrede.,NaN,"1, 2",1,https://alex.onb.ac.at/cgi-content/alex?aid=sp...
1,image_00000005.tif,2,"Forterhebung der Steuern und Abgaben, dann Bes...",Regierungsvorlage,3,3,https://alex.onb.ac.at/cgi-content/alex?aid=sp...
2,image_00000005.tif,3,Finanzgesetz und Staatsooranschlag für das Jah...,Regierungsvorlage,4—30,4,https://alex.onb.ac.at/cgi-content/alex?aid=sp...
3,image_00000005.tif,4,Eröffnung von Nachtragscrediten für das Jahr 1...,Regierungsvorlage,31,31,https://alex.onb.ac.at/cgi-content/alex?aid=sp...
4,image_00000005.tif,5,Ermächtigung zu einer Creditsoperation durch B...,Regierungsvorlage und Motive,32—43,32,https://alex.onb.ac.at/cgi-content/alex?aid=sp...
...,...,...,...,...,...,...,...
352,image_00000038.tif,353,"Aufhebung des Gesetzes vom 30. Juni 1868, RGBl...",Beschluß des Abge ordnetenhauses,3075,3075,https://alex.onb.ac.at/cgi-content/alex?aid=sp...
353,image_00000039.tif,354,Antrag des Abgeordneten Dr. Roser wegen Er las...,Bericht,3076—3082,3076,https://alex.onb.ac.at/cgi-content/alex?aid=sp...
354,image_00000039.tif,355,Polizeistrafrechtliche Bestimmungen wider Arbe...,Beschluß des Abge ordnetenhauses,3083 — 3086,3083,https://alex.onb.ac.at/cgi-content/alex?aid=sp...
355,image_00000039.tif,356,a) Hauptübersicht über den Stand der gesammten...,Vorlage der Staats schulden - Control- commission,3087—3093,3087,https://alex.onb.ac.at/cgi-content/alex?aid=sp...


In [190]:
#Verbinden der values von Titel und Typ
df_Beilagen_v3['Titel'] = df_Beilagen_v3['Typ'].astype(str) + ': ' + df_Beilagen_v3['Titel'].astype(str)

In [191]:
df_Beilagen_v3.head()

,Image_No,Beilagen_No,Titel,Typ,Seite,Seite_cleaned,PageNumber
0,image_00000005.tif,1,nan: Allerhöchste Thronrede.,NaN,"1, 2",1,https://alex.onb.ac.at/cgi-content/alex?aid=sp...
1,image_00000005.tif,2,Regierungsvorlage: Forterhebung der Steuern un...,Regierungsvorlage,3,3,https://alex.onb.ac.at/cgi-content/alex?aid=sp...
2,image_00000005.tif,3,Regierungsvorlage: Finanzgesetz und Staatsoora...,Regierungsvorlage,4—30,4,https://alex.onb.ac.at/cgi-content/alex?aid=sp...
3,image_00000005.tif,4,Regierungsvorlage: Eröffnung von Nachtragscred...,Regierungsvorlage,31,31,https://alex.onb.ac.at/cgi-content/alex?aid=sp...
4,image_00000005.tif,5,Regierungsvorlage und Motive: Ermächtigung zu ...,Regierungsvorlage und Motive,32—43,32,https://alex.onb.ac.at/cgi-content/alex?aid=sp...


In [193]:
# Drop the redundant Typ column from the merge
df_Beilagen_v3.drop(columns=['Typ'], inplace=True)

In [195]:
df_Beilagen_v3.to_excel('beilagen_Session7_tables_v4.xlsx', index=False)